pybaseball github with install instructions: https://github.com/jldbc/pybaseball <br>
Documentation for pybaseball library: https://github.com/jldbc/pybaseball/tree/master/docs

In [1]:
import pybaseball
import pandas as pd
from collections import Counter

In [2]:
def pitcher(last_name, first_name):
    pitcher = pybaseball.playerid_lookup(last_name, first_name)
    return pitcher

In [3]:
def games_pitched_in_timeframe(first_name, last_name, start_date, end_date):
    l_name = last_name
    f_name = first_name
    pitcher_data = pitcher(l_name, f_name)
    pitcher_id = str(pitcher_data['key_mlbam'][0])
    pitching_data = pybaseball.statcast_pitcher(start_date, end_date, pitcher_id)
    return list(Counter(pitching_data['game_date']))

In [4]:
def pitcher_game_number(date, team):
    months = {'Jan' : '01', 'Feb' : '02', 'Mar' : '03', 'Apr' : '04', 'May' : '05', 'Jun' : '06',
          'Jul' : '07', 'Aug': '08', 'Sep' : '09', 'Oct' : '10', 'Nov' : '11', 'Dec' : '12'}
    
    game_date = date.split('-')
    year = game_date[0]
    game_date = game_date[1:]
    game_date = '-'.join(game_date)
    
    pitching_logs = pybaseball.team_game_logs(year, team, "pitching")
    test = list(pitching_logs['Date'])
    test = [i.split() for i in test]
    x = 0
    for ind in test:
        ind[0] = months.get(ind[0])
        new_string = '-'.join(ind)
        test[x] = new_string
        x+=1
    pitching_logs['Date'] = test
    game = pitching_logs.loc[pitching_logs['Date'] == game_date].reset_index()
    data = pybaseball.statcast(date, date)
    if game['Home'][0] == True:
        test = data.loc[data['away_team'] == 'LAD']
        game_pk = list(test['game_pk'])
        return game_pk[0]
    else:
        test = data.loc[data['home_team'] == 'LAD']
        game_pk = list(test['game_pk'])
        return game_pk[0]

In [5]:
def pitcher_game_complete_data(game_id, pitcher_id):
    data = pybaseball.statcast_single_game(game_id).reset_index(drop = True)
    pitcher_data = pd.DataFrame()
    for x in range(len(data)):
        if data['pitcher'][x] == pitcher_id:
            pitcher_data = pitcher_data.append(data.iloc[[x]])
    pitcher_data = pitcher_data.reset_index(drop = True)
    return pitcher_data

In [6]:
def pitch_summary(game_data):
    data = Counter(game_data['pitch_type'])
    return data

In [109]:
def ball_and_strike(game_data):
    balls = pd.DataFrame(columns = ['pitch_type', 'description'])
    strikes = pd.DataFrame(columns = ['pitch_type', 'description'])
    live = pd.DataFrame(columns = ['pitch_type', 'description'])
    k = 0
    for x in range(len(game_data)):
        if game_data['description'][x] == 'ball':
            details = game_data[['pitch_type','description']].iloc[[x]]
            balls = balls.append(details)
        elif game_data['description'][x] == 'hit_into_play':
            details = game_data[['pitch_type','description']].iloc[[x]]
            live = live.append(details)
        else:
            details = game_data[['pitch_type','description']].iloc[[x]]
            strikes = strikes.append(details)
    return balls, strikes, live

In [66]:
def main(first_name, last_name, start_date, end_date, team):
    year = start_date.split('-')
    year = int(start_date[0])
    
    desired_pitcher = pitcher(last_name, first_name)
    desired_pitcher_id = desired_pitcher['key_mlbam'][0]
    
    games = games_pitched_in_timeframe(first_name, last_name, start_date, end_date)
    
    game_id = pitcher_game_number(games[0], team)
    game_pitch_data = pitcher_game_complete_data(game_id, desired_pitcher_id)
    print(pitch_summary(game_pitch_data))
    print(ball_and_strike(game_pitch_data))
#     return game_pitch_data

In [110]:
main('clayton','kershaw','2017-06-29','2017-06-29', 'LAD')

Gathering Player Data
This is a large query, it may take a moment to complete


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.11s/it]


Counter({'FF': 38, 'SL': 31, 'CU': 23, 'CH': 1})
(   pitch_type description
1          SL        ball
3          FF        ball
9          FF        ball
10         SL        ball
16         FF        ball
20         CU        ball
26         FF        ball
30         CU        ball
41         CU        ball
47         SL        ball
50         FF        ball
51         FF        ball
57         FF        ball
62         SL        ball
63         SL        ball
71         CU        ball
79         SL        ball
80         SL        ball
85         FF        ball
89         CU        ball
92         FF        ball,    pitch_type              description
0          SL  swinging_strike_blocked
2          CU                     foul
4          CU            called_strike
5          SL          swinging_strike
7          FF                     foul
8          FF          swinging_strike
11         SL            called_strike
13         SL            called_strike
14         SL          swi